# **Modelagem, Validação e Inferência**

## 1. Objetivo

Após uma fase intensiva de Análise Exploratória e Engenharia de _Features_, que nos proporcionou um profundo entendimento do negócio e um _dataset_ enriquecido, chegamos à etapa final e decisiva deste projeto: a **construção da solução preditiva**.

O objetivo deste _notebook_ é desenvolver e validar rigorosamente um modelo de _Machine Learning_ capaz de prever a **probabilidade de inadimplência** de um cliente. O entregável final será o arquivo `submissao_case.csv`, contendo as predições para a base de teste fornecida.

## 2. A Nossa Estratégia de Modelagem e Validação

Para garantir que nosso modelo seja não apenas performático, mas também **robusto** e **confiável**, adotaremos uma estratégia metodológica clara.

### 2.1 O Desafio: Avaliação vs. Inferência Final

O _case_ nos fornece dois conjuntos de dados de pagamentos:
* `base_pagamentos_desenvolvimento.csv`: Nosso dataset de **desenvolvimento**, para o qual temos o histórico completo, incluindo as datas de pagamento que nos permitem calcular a inadimplência.
* `base_pagamentos_teste.csv`: Nosso dataset de **teste final** ("produção"), para o qual **não temos o desfecho**.

A questão central é: **Como podemos avaliar e otimizar nosso modelo se não temos o gabarito da base de teste final?**

A resposta está na **validação local rigorosa**. Antes de aplicarmos nosso modelo ao desafio final, precisamos simular o "dia do exame" usando apenas os dados de desenvolvimento.

### 2.2 Plano de Ação

Nosso fluxo de trabalho será dividido em três fases macro:

**Fase 1: Preparação e Validação do Modelo (Ambiente de Desenvolvimento)**

1.  **Geração dos Dados:** Utilizaremos nossas funções customizadas em `src/` para processar a base de desenvolvimento e gerar o dataset `X_dev` (features) e `y_dev` (alvo `INADIMPLENTE`).

2.  **Seleção do Modelo via Validação Cruzada (Cross-Validation):** Em vez de um simples `train_test_split`, empregaremos a **Validação Cruzada Estratificada**. Esta técnica, mais consistennte, divide os dados de desenvolvimento em múltiplos "_folds_" (partes), treinando e testando o modelo várias vezes. Isso nos dá uma estimativa muito mais estável e confiável da sua performance real.
    * **Modelo Baseline:** Avaliaremos a performance média de uma **Regressão Logística**.
    * **Modelo Avançado:** Avaliaremos a performance média de um modelo mais sofisticado, como o **XGBoost**.

3.  **Seleção do Campeão:** Com base na média das métricas da Validação Cruzada (foco em AUC-ROC e Recall), selecionaremos o algoritmo vencedor.

**Fase 2: Treinamento Final**

4.  **Retreinamento:** Uma vez escolhido o melhor modelo (ex: XGBoost) e seus melhores parâmetros, iremos **retreiná-lo utilizando 100% dos nossos dados de desenvolvimento** (`X_dev` e `y_dev`). O objetivo é dar ao nosso modelo final todo o conhecimento possível antes de enfrentar o desafio real.

**Fase 3: Geração da Submissão Final**

5.  **Preparação dos Dados de Teste:** Aplicaremos **exatamente os mesmos passos de processamento e engenharia de features** na base `base_pagamentos_teste.csv`.
6.  **Inferência:** Usaremos o nosso modelo final, treinado na Fase 2, para fazer as predições na base de teste preparada, utilizando o método `.predict_proba()` para obter as probabilidades.
7.  **Criação do Arquivo Final:** Montaremos o arquivo `submissao_case.csv` no formato exato especificado no desafio.

---

Este plano metodológico garante que cada passo seja deliberado, justificável e que o resultado final seja fruto de um processo de validação rigoroso, pois nosso intuito é definir uma estratégia de trabalho de alto nível para uma tarefa delicada.